### Maximal Marginal Relevance (MMR) Kya Hai?

**MMR** ek technique hai jo aapko examples select karne mein madad karti hai. Iska maksad ye hai ke jo examples aap select kar rahe hain, wo aapas mein similar hone ke saath-saath thodi diversity bhi rakhein, yani sab kuch ek jaisa na ho.

### Kaise Kaam Karta Hai?

1. **Examples**: Aapke paas kuch examples hain, jahan har example ek input aur uska output hota hai. Jaise:
   - Input: "happy", Output: "sad"
   - Input: "tall", Output: "short"

2. **Similarity**: MMR ko samajhne ke liye yeh zaroori hai ke hum examples ki similarity ko measure karein. Ye similarity samajhne ke liye hum embeddings ka istemal karte hain. Embeddings basically ek tarah ki numerical representation hoti hain jo humein batati hain ke do words ya phrases kitne similar hain.

3. **Diversity**: Jab MMR examples ko select karta hai, to wo ye dekhta hai ke jo naye examples wo select kar raha hai, wo pehle se selected examples se zyada similar na hon. Is tarah se wo diversity ko bhi maintain karta hai.

### Example Kaise Kaam Karta Hai:

1. **MMR Example Selector Ka Istemal**: Aap MMR example selector ko use karte hain. Isme aapko pehle examples ki list deni hoti hai, phir wo unme se sabse relevant aur diverse examples select karta hai.

2. **Input dena**: Jab aap input dete hain, for example "worried", MMR dekhta hai ke kaunse examples is input ke liye sabse behtar hain.

### Semantic Similarity Selector:

Agar aap sirf similarity dekhna chahte hain, to aap **Semantic Similarity Example Selector** use karte hain. Yeh sirf un examples ko select karega jo input se zyada similar hain, bina diversity ki parwah kiye.

### Summary:

- **MMR**: Similar aur diverse examples select karta hai.
- **Semantic Selector**: Sirf similar examples select karta hai.

Agar aapko ab bhi kuch samajh nahi aaya ya koi aur sawal hai, to aap puch sakte hain!

Diversity ka matlab hai ke selected examples mein variety ho, yaani wo ek dusre se itne similar na hon ke unka output ya meaning bilkul ek jaisa ho. Jab aap examples ko select karte hain, to agar aap har baar sirf wohi examples chunte hain jo ek jaisa hain, to aapke paas koi variety nahi hogi.

### MMR mein Diversity Ki Importance:

1. **Variety**: MMR ka maksad ye hai ke aapke selected examples mein kuch variety ho, taake jab aap kisi nayi input par kaam karte hain, to aapke paas different perspectives ya options hon.

2. **Avoiding Redundancy**: Agar aap sirf similar examples lete hain, to aap kaam ko itna effective nahi bana sakte. Jaise agar aapke paas sirf "happy" aur "joyful" hon, to inka output bhi same hoga. Lekin agar aap "happy" aur "sad" lete hain, to ye ek dusre se different hain aur aapko better understanding de sakte hain.

3. **Enhanced Learning**: Diverse examples aapko better learning experience dete hain. Jab aapke paas different examples hote hain, to aap unhe compare karke seekhte hain ke kaise alag-alag inputs ke liye alag outputs ho sakte hain.

### Example se Samjhana:

Agar aap sirf "happy" aur "joyful" ke examples lete hain:
- Input: "happy" → Output: "joyful"
- Input: "happy" → Output: "joyful"

Yahan par aapke paas koi diversity nahi hai, dono outputs bilkul same hain.

Lekin agar aap "happy" aur "sad" lete hain:
- Input: "happy" → Output: "sad"
- Input: "windy" → Output: "calm"

Yahan par aapko diversity milti hai, kyunke aapke selected examples alag-alag meanings aur contexts ko represent kar rahe hain.

### Conclusion:

Diversity ensure karta hai ke aapka learning process aur outputs zyada informative aur useful hon. MMR isliye diversity par focus karta hai, taake aapko behtar results milein aur aap kisi bhi input par alag-alag perspectives se soch sakein.

In [8]:
from langchain_core.prompts import ChatPromptTemplate, FewShotPromptTemplate, PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.example_selectors import SemanticSimilarityExampleSelector,  MaxMarginalRelevanceExampleSelector
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnableSequence

from dotenv import load_dotenv
import os

load_dotenv()

# Load the API key from .env file
gemini_api_key = os.getenv("GEMINI_API_KEY")
os.environ["GEMINI_API_KEY"] = gemini_api_key

# Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=gemini_api_key,  # Corrected the variable usage
    temperature=0.2,
    verbose=True,
)

# Examples for few-shot prompting
examples = [
    {"input": "What is the capital of France?", "output": "The capital of France is Paris."},
    {"input": "Who wrote the play 'Romeo and Juliet'?", "output": "William Shakespeare wrote 'Romeo and Juliet'."},
    {"input": "What is the boiling point of water in Celsius?", "output": "The boiling point of water is 100°C."},
    {"input": "Who is the current CEO of Tesla?", "output": "Elon Musk is the current CEO of Tesla."}
]

# Embeddings setup
embeddings = GoogleGenerativeAIEmbeddings(
    google_api_key=gemini_api_key,
    model="models/text-embedding-004",
)


# Documents to store in vector database
documents = [Document(page_content=example['input'], metadata={"input": example["input"], "output": example["output"]})
             for example in examples]

# Create a vector store using FAISS
vector_store = FAISS.from_documents(documents, embeddings)

# Example selector using SemanticSimilarityExampleSelector
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vector_store,  
    k=2,
)

# Example selector using MaxMarginalRelevanceExampleSelector
# example_selector = MaxMarginalRelevanceExampleSelector(
#     vectorstore=vector_store,  
#     k=2,
# )

# Create the prompt template
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Dynamic few-shot prompt template
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {input}\nOutput:",
    input_variables=["input"],
    # verbose=True,
)
print(dynamic_prompt.invoke(
    {
        "input": "what is standard temprature in the earth?",


    }
))

# # Create the LLMChain
# # Example code using RunnableSequence
# llm_chain = RunnableSequence(dynamic_prompt | llm)


# # Run the chain with an example input
# chain_input = {"input": "who is the CEO  of Google?"}


# chain_output = llm_chain.invoke(chain_input)

# # Print the output of the LLMChain
# print(chain_output.content)


text='Give the antonym of every input\n\nInput: What is the boiling point of water in Celsius?\nOutput: The boiling point of water is 100°C.\n\nInput: What is the capital of France?\nOutput: The capital of France is Paris.\n\nInput: what is standard temprature in the earth?\nOutput:'
